<a href="https://colab.research.google.com/github/avinashdhakne/Covid-19-Information-Chatbot-using-Deep-learning/blob/main/1_data_collection_processsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Collection**

In [ ]:
# Importing the libraries to scrap the data form website
import requests
import bs4

# requesting the page to scrap the data
page = requests.get("https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub")
# conver the html code to text format
soup = bs4.BeautifulSoup(page.text,"lxml")
# find the tag where required data is present
data = soup.find("div", class_="sf-list-vertical sf-list-vertical__nodate")

# creating string to link
str_link = "https://www.who.int"
# Creating list to store the link of each FAQ page
links = []
# Scrape the link for each FAQ page
for a in data.find_all('a', href=True):
    link = str_link+a['href']
    links.append(link)

# creating intitiating tag for each intent
tag_code = 'tag#'
tag_count = 0
# create the dictionry fordataset
dataset = {"Intents":[]}

In [ ]:
# Getting the data form links
for link in links:
  
  # If the link is emplty then end the loop
  if link == '':
    break

  # request the FAQ page to scrape the data 
  page = requests.get(link)
  # Convert HTML code to text format
  soup = bs4.BeautifulSoup(page.text,"lxml")
  # Search the tag where required data is presnt
  data = soup.find_all("div", class_="sf-accordion__panel")

  # creating unique tag for each intent
  for text in data:
    tag_count += 1
    tag = tag_code + str(tag_count)

    # scrap the question from the webpage
    question = text.find('span',itemprop = "name").text
    responces = []

    # Scrape the answer form the webpage
    responce = text.find_all('p')
    for para in responce:
      clear_responce = para.get_text()
      responces.append(clear_responce)

    # Save all the data to dictionary format
    new_dic = {"tag":tag,"patterns":[question],"responses":[''.join(responces)],"context":[]}
    # append the scapped data to dataset
    dataset['Intents'].append(new_dic)

# Check the length of the dataset
len(dataset['Intents'])

515

# **Data Merging**

In [ ]:
# importing the manual dataset
from google.colab import files
files.upload()

Saving my_dataset.json to my_dataset.json


{'my_dataset.json': b'{\r\n    "Intents": [\r\n        {\r\n            "tag": "Basics_of_covid",\r\n            "patterns": [\r\n                "What is COVID-19?",\r\n                "What is Coronavirus?",\r\n                "Novel coronavirus",\r\n                "what is COVID-19",\r\n                "Explain corona virus",\r\n                "Tell me about covid",\r\n                "tell me about corona virus"\r\n            ],\r\n            "responses": [\r\n                "COVID-19, otherwise known as coronavirus disease 2019, is a new infectious disease caused by a previously unknown virus called SARS-CoV-2. The virus is part of a family of coronaviruses which are responsible for lots of different illnesses from the common cold to the flu. But this new strain can be more severe in some people.",\r\n                "COVID-19 is an infectious respiratory illness caused by a newly discovered coronavirus called SARS-CoV-2. \xe2\x80\x98CO\xe2\x80\x99 stands for corona, \xe2\x80

In [ ]:
# importing library to read the json file
import json

# Open the json file
with open("/content/my_dataset.json") as file1:
  first_file = json.load(file1)
len(first_file["Intents"])

56

In [ ]:
# mearge the scrapped and manual dataset
for intent in first_file["Intents"]:
  dataset["Intents"].append(intent)
len(dataset['Intents'])

571

# **Data Augmentation**

In [ ]:
# download the parrot library for data augmentation
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git
# importing the library
from parrot import Parrot
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-grynj0rh
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-grynj0rh


In [ ]:
# augmentation of the data 
count = 0
for intent in dataset["Intents"]:
  for pattern in intent['patterns']:
    para_phrases = parrot.augment(input_phrase = pattern, use_gpu=True)
    if para_phrases != "": 
      print(para_phrases)
    else:
      continue
  count+=1
  print("-----**********---intent no.",count,"is completed---**********-----")
  for phrase in para_phrases:
    intent['patterns'].append(str(phrase))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2268: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


[('what does covid-19 mean?', 91)]
-----**********---intent no. 1 is completed---**********-----
[('what are signs and symptoms of covid-19?', 94), ('tell me the symptoms of covid-19?', 93), ('what are symptoms of covid-19?', 92), ('can you list the symptoms of covid-19?', 92), ('can you list the symptoms of covid-19? ', 91)]
-----**********---intent no. 2 is completed---**********-----
[('how does covid-19 affect the people who have it?', 106), ('what happens to covid-19 patients?', 103), ('what happens if the symptoms get covid-19?', 101), ('what happens if you get covid-19?', 99), ('what happens to people with covid-19?', 94)]
-----**********---intent no. 3 is completed---**********-----
[('who is at greatest risk of having a covid-19 infection?', 111), ('who is the most at risk from covid-19?', 108), ('who is most at risk for covid-19?', 108), ('who is most at risk for severe covid-19?', 103), ('who is most at risk of severe covid-19?', 101), ('who is most at risk of the severe ill

In [ ]:
# augmenting the data again to provid more robustness to the model
count = 0
for intent in dataset["Intents"]:
  for pattern in intent['patterns']:
    para_phrases = parrot.augment(input_phrase=pattern, use_gpu=True)
    if para_phrases != "": 
      print(para_phrases)
    else:
      continue
  count+=1
  print("-----**********---intent no.",count,"is completed---**********-----")
  for phrase in para_phrases:
    intent['patterns'].append(str(phrase))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2268: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


[('\r\n                                        What is COVID-19?\r\n                                    ', 0)]
[('what exactly is covid-19?', 26), ('what do you think covid-19 means?', 24), ('what does covid-19 mean?', 17), ("what does covid-19 mean?'", 16)]
-----**********---intent no. 1 is completed---**********-----
[("what's the symptoms of covid-19?", 92), ('what are symptoms of covid-19?', 92), ('can you list the symptoms of covid-19?', 92)]
[('what are the symptoms of covid-19?', 26), ('list the signs and symptoms of covid-19?', 21), ('what are signs and symptoms of covid-19?', 17), ("what are signs and symptoms of covid-19?'", 16)]
[('do you know the symptoms of covid-19? 93', 20), ("tell me the symptoms of covid-19?'", 16)]
[('what are symptoms of vid-19?', 19), ('what are the symptoms of covid-19?', 19), ("what's the symptom of covid-19? 92", 18), ('what are symptoms of covid-19?', 17)]
[("can you list the symptoms of covi-19?'", 17), ('can you list the symptoms of covid-19?'

In [ ]:
# save the augmented data to json format
json_object = json.dumps(dataset)
  
# download the data in json format
with open("final_data.json", "w") as outfile:
    outfile.write(json_object)